# シンプルなFC層を用いたモデル

### 方針

・条件付き確率（遷移確率）と尤度の和から次の尤度を求める

### データの生成

・データの用意

In [10]:
import numpy as np

# 依存関係の行列 A (問題の依存関係)
A = np.array([
    [0, 0, 0, 0, 0],  # 初期状態
    [1, 0, 0, 0, 0],  # 問題1は初期状態のみに依存
    [0, 1, 0, 0, 0],  # 問題2は問題1に依存
    [0, 0, 1, 0, 0],  # 問題3は問題2に依存
    [0, 0, 1, 1, 0]   # 問題4は問題2、問題3に依存
])


# 遷移確率を計算する関数（そのまま使用）
def calculate_transition_probabilities(A, X):
    n = len(X)
    raw_probabilities = np.zeros(n)  # 遷移確率の元となる値
    
    # 不正解の問題に対して遷移確率を計算
    for i in range(n):
        if X[i] == 0:  # 不正解の問題のみ計算
            required_problems = A[i, :]  # i番目の問題に必要な依存関係
            
            # # 依存関係がすべて0であるかを確認
            # if np.all(required_problems == 0):
            #     raw_probabilities[i] =  10  # 必要なら、この問題の遷移確率を設定
            #     continue
            
            solved_problems = X * required_problems  # 実際に解けた問題
            
            num_required = np.sum(required_problems)  # 必要な問題の数
            num_solved = np.sum(solved_problems)      # 実際に解けた問題の数
            
            if num_required > 0:
                raw_probabilities[i] = np.exp(num_solved / num_required)
    
    # 総和で割って正規化
    total_sum = np.sum(raw_probabilities)  # expの総和
    if total_sum > 0:  # 総和が0でなければ正規化
        probabilities = raw_probabilities / total_sum
    else:
        probabilities = raw_probabilities  # 総和が0ならそのまま
    
    return probabilities

In [11]:
# 教師データセットを生成する関数
def generate_training_data(A, initial_X, num_correct_problems, num_data_per_step):
    n = len(initial_X)  # 問題数
    dataset = []
    
    # 各ステップでデータを生成
    for i in range(1, num_correct_problems + 1):  # 1問以上の正解を対象にする
        for j in range(num_data_per_step):  # 各ステップごとにデータ数
            X = initial_X.copy()  # 初期状態からスタート
            input_X = X.copy()    # 初期状態を保持
            
            # i問正解させる
            for k in range(i):
                probabilities = calculate_transition_probabilities(A, X)
                
                if np.sum(probabilities) > 0:  # 正規化された確率がある場合
                    # 確率に基づいて次に正解させる問題を選択
                    next_correct_problem = np.random.choice(n, p=probabilities)
                    X[next_correct_problem] = 1  # 選ばれた問題を正解に遷移させる
            
            # 初期状態と1ステップ後の状態の差分を教師データとして使用
            target_Y = (X - input_X).clip(min=0)  # 0から1に変わった部分のみを1、他は0
            
            # 初期状態（入力）と差分（教師データ）のペアを保存
            dataset.append((input_X.copy(), target_Y.copy()))  # (入力データ, 教師データ)
    
    return dataset


In [12]:
# データセット生成
num_correct_problems = 5  # 初期状態から4問まで正解
num_data_per_step = 100     # 各ステップごとに生成するデータ数

# 生徒の回答状況 X (1が正解、0が不正解)
# 初期状態は全て不正解
X_init = np.array([1, 0, 0, 0, 0])

dataset = generate_training_data(A, X_init, num_correct_problems, num_data_per_step)

# 結果を表示
print("生成されたデータセット:")
print(dataset)

生成されたデータセット:
[(array([1, 0, 0, 0, 0]), array([0, 0, 0, 0, 1])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 1, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 0, 0, 1])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 1, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 1, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 0, 0, 1])), (array([1, 0, 0, 0, 0]), array([0, 0, 0, 1, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0

In [13]:
import itertools
from collections import defaultdict

num_questions = 4
num_questions = num_questions + 1 # 初期状態を追加

# 問題数5問に対する全ての可能な状態 (2^5 = 32状態)
states = list(itertools.product([0, 1], repeat=num_questions))
print(f"Number of States: {len(states)}\n")

# 各状態（ノード）に生徒を配置する (状態ごとの生徒数をカウント)
state_counts = defaultdict(int)

# # 生徒ごとのテスト結果を元に状態に生徒をカウント
# for result in dataset:
#     state_tuple = tuple(map(int, result))  # np.int64をint型に変換
#     state_counts[state_tuple] += 1

# # 各状態の生徒数を表示
# for state, count in state_counts.items():
#     formatted_state = ''.join(map(str, state))  # 状態を'01010'のような形式に変換
#     print(f"State [{formatted_state}]: {count} students")

Number of States: 32



In [14]:
# import networkx as nx
# import matplotlib.pyplot as plt
# import numpy as np

# # グラフの作成
# G = nx.Graph()

# # ノードの追加
# for state in states:
#     G.add_node(state, count=state_counts[state])

# # エッジの追加
# for state in states:
#     for i in range(num_questions):
#         next_state = list(state)
#         if next_state[i] == 0:
#             next_state[i] = 1
#             G.add_edge(state, tuple(next_state))

# # カスタムレイアウト関数
# def custom_layout(G, num_questions):
#     pos = {}
#     for state in G.nodes():
#         level = sum(state)
#         # 各レベルでのノードの数を数える
#         level_count = sum(1 for node in G.nodes() if sum(node) == level)
#         # このレベルでの現在のノードの位置を計算
#         level_position = sum(state[i] * 2**(num_questions-i-1) for i in range(num_questions))
#         x = level_position / (2**num_questions - 1)  # x座標を0~1の範囲に正規化
#         y = level / num_questions  # y座標を0~1の範囲に正規化
#         pos[state] = (x, y)
#     return pos

# # カスタムレイアウトの適用
# pos = custom_layout(G, num_questions)

# # プロットの設定
# plt.figure(figsize=(18, 6))  # 縦長に変更

# # エッジの描画
# nx.draw_networkx_edges(G, pos, alpha=0.2)

# # ノードの描画
# node_sizes = [max(300, min(3000, G.nodes[node]['count'] * 50)) for node in G.nodes()]
# node_colors = [G.nodes[node]['count'] for node in G.nodes()]
# nodes = nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, cmap=plt.cm.YlOrRd)

# # ノードのラベル描画
# nx.draw_networkx_labels(G, pos, {node: ''.join(map(str, node)) for node in G.nodes()}, font_size=8)

# # カラーバーの追加
# plt.colorbar(nodes, label='Number of Students')

# plt.title('Vertical Network Graph of Test Results')
# plt.axis('off')
# plt.tight_layout()

# # グラフの表示
# plt.show()

# # 各状態の学生数を表示
# for state, count in sorted(state_counts.items(), key=lambda x: sum(x[0])):
#     print(f"State {''.join(map(str, state))}: {count} students")

## モデル

### FCモデル

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

class Model(nn.Module):
    def __init__(self, num_questions):
        super(Model, self).__init__()
        self.fc = nn.Linear(num_questions, num_questions)  # 全結合層

    def forward(self, x):
        x = self.fc(x)  # 全結合層の適用
        x = F.softmax(x, dim=1)  # ソフトマックスを適用
        return x

In [19]:
# データセットの生成
num_correct_problems = 5  # 初期状態から4問まで正解
num_data_per_step = 100     # 各ステップごとに生成するデータ数

# 生徒の回答状況 X (1が正解、0が不正解)
# 初期状態は全て不正解
X_init = np.array([1, 0, 0, 0, 0])

training_data = generate_training_data(A, X_init, num_correct_problems, num_data_per_step)

# PyTorch テンソルに変換
train_X = torch.tensor([input_data for input_data, _ in training_data], dtype=torch.float32)
train_Y = torch.tensor([target_data for _, target_data in training_data], dtype=torch.float32)

# モデル、損失関数、最適化関数の設定
model = Model(num_questions=5)  # 5問の問題を扱うモデル
criterion = nn.CrossEntropyLoss()  # クロスエントロピー損失
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adamオプティマイザ

[(array([1, 0, 0, 0, 0]), array([0, 0, 0, 0, 1])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 1, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 0, 1, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 0, 1, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 0, 0, 1])), (array([1, 0, 0, 0, 0]), array([0, 0, 1, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 0, 0, 1])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 0, 0, 1])), (array([1, 0, 0, 0, 0]), array([0, 0, 1, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 1, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 0, 0, 1])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 1, 0, 0, 0])), (array([1, 0, 0, 0, 0]), array([0, 0, 0, 1, 0])),

In [17]:
# 学習ループ
num_epochs = 100000  # エポック数
for epoch in range(num_epochs):
    model.train()  # モデルを訓練モードに
    optimizer.zero_grad()  # 勾配の初期化
    
    # モデルの予測
    outputs = model(train_X)
    
    # 損失の計算
    loss = criterion(outputs, train_Y)
    
    # バックプロパゲーションとパラメータの更新
    loss.backward()
    optimizer.step()
    
    # 100エポックごとに損失を表示
    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 学習結果の確認
print("Training complete!")

Epoch [100/100000], Loss: 4.3735
Epoch [200/100000], Loss: 4.3646
Epoch [300/100000], Loss: 4.3622
Epoch [400/100000], Loss: 4.3612
Epoch [500/100000], Loss: 4.3606
Epoch [600/100000], Loss: 4.3603
Epoch [700/100000], Loss: 4.3600
Epoch [800/100000], Loss: 4.3599
Epoch [900/100000], Loss: 4.3598
Epoch [1000/100000], Loss: 4.3597
Epoch [1100/100000], Loss: 4.3596
Epoch [1200/100000], Loss: 4.3595
Epoch [1300/100000], Loss: 4.3595
Epoch [1400/100000], Loss: 4.3595
Epoch [1500/100000], Loss: 4.3594
Epoch [1600/100000], Loss: 4.3594
Epoch [1700/100000], Loss: 4.3594
Epoch [1800/100000], Loss: 4.3594
Epoch [1900/100000], Loss: 4.3594
Epoch [2000/100000], Loss: 4.3593
Epoch [2100/100000], Loss: 4.3593
Epoch [2200/100000], Loss: 4.3593
Epoch [2300/100000], Loss: 4.3593
Epoch [2400/100000], Loss: 4.3593
Epoch [2500/100000], Loss: 4.3593
Epoch [2600/100000], Loss: 4.3593
Epoch [2700/100000], Loss: 4.3593
Epoch [2800/100000], Loss: 4.3593
Epoch [2900/100000], Loss: 4.3593
Epoch [3000/100000], Lo

## テスト

In [24]:
test = np.array([[1, 1, 1, 0, 0]])

# numpy配列をtorch.Tensorに変換
test_tensor = torch.tensor(test, dtype=torch.float32)

# モデルに入力を渡して出力を得る
output = model(test_tensor)
print("Output:", output)

Output: tensor([[0.0974, 0.0689, 0.3569, 0.2350, 0.2418]], grad_fn=<SoftmaxBackward0>)


・図示

In [18]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

# グラフの作成
G = nx.Graph()

# ノードの追加
for state in states:
    G.add_node(state, likelihood=likelihoods[state])

# エッジの追加
for state in states:
    for i in range(num_questions):
        next_state = list(state)
        if next_state[i] == 0:
            next_state[i] = 1
            G.add_edge(state, tuple(next_state))

# カスタムレイアウト関数
def custom_layout(G, num_questions):
    pos = {}
    for state in G.nodes():
        level = sum(state)
        # 各レベルでのノードの数を数える
        level_count = sum(1 for node in G.nodes() if sum(node) == level)
        # このレベルでの現在のノードの位置を計算
        level_position = sum(state[i] * 2**(num_questions-i-1) for i in range(num_questions))
        x = level_position / (2**num_questions - 1)  # x座標を0~1の範囲に正規化
        y = level / num_questions  # y座標を0~1の範囲に正規化
        pos[state] = (x, y)
    return pos

# カスタムレイアウトの適用
pos = custom_layout(G, num_questions)

# 尤度の合計を計算
total_likelihood = sum(likelihoods.values())

# 各ノードの確率を計算して追加
for state in G.nodes():
    likelihood = G.nodes[state]['likelihood']
    probability = likelihood / total_likelihood if total_likelihood > 0 else 0
    G.nodes[state]['probability'] = probability

# プロットの設定
plt.figure(figsize=(18, 6))  # 縦長に変更

# エッジの描画
nx.draw_networkx_edges(G, pos, alpha=0.2)

# ノードの描画 (尤度に基づく)
# ノードサイズと色を尤度に基づいて設定
node_sizes = [max(300, min(3000, G.nodes[node]['likelihood'] * 3000)) for node in G.nodes()]
node_colors = [G.nodes[node]['likelihood'] for node in G.nodes()]
nodes = nx.draw_networkx_nodes(G, pos, node_size=node_sizes, node_color=node_colors, cmap=plt.cm.Blues)

# ノードのラベル描画 (状態・尤度・確率)
labels = {node: f"{''.join(map(str, node))}\nL:{G.nodes[node]['likelihood']:.2f}, P:{G.nodes[node]['probability']:.2%}" for node in G.nodes()}
nx.draw_networkx_labels(G, pos, labels, font_size=8)

# カラーバーの追加
plt.colorbar(nodes, label='Likelihood')

plt.title('Vertical Network Graph of Test Results (Likelihood and Probability)')
plt.axis('off')
plt.tight_layout()

# グラフの表示
plt.show()

# 各状態の尤度と確率を表示
for state, likelihood in sorted(likelihoods.items(), key=lambda x: sum(x[0])):
    probability = likelihood / total_likelihood if total_likelihood > 0 else 0
    print(f"State {''.join(map(str, state))}: Likelihood = {likelihood:.4f}, Probability = {probability:.4%}")


NameError: name 'likelihoods' is not defined

### 学習

### テスト